In [1]:
# --- Core imports ---
import os, pickle, time
import numpy as np
import tensorflow as tf
from pathlib import Path

# --- Keras imports ---
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
)

print("✅ TensorFlow version:", tf.__version__)

# --- GPU setup ---
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
    print("✅ GPU detected and configured")
else:
    print("⚠️ Running on CPU — training may be slow")


✅ TensorFlow version: 2.19.0
⚠️ Running on CPU — training may be slow


In [2]:
from google.colab import drive
drive.mount("/content/drive")

# Change this to your dataset location
dataset_root = Path("/content/drive/MyDrive/24_images")

train_dir = dataset_root / "train"
val_dir   = dataset_root / "test"

assert train_dir.exists(), f"❌ Train folder not found: {train_dir}"
assert val_dir.exists(), f"❌ Test folder not found: {val_dir}"

print(f"✅ Dataset found: {dataset_root}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dataset found: /content/drive/MyDrive/24_images


In [3]:
IMG_SIZE = (128, 128)     # smaller, faster for custom CNN
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

emotion_labels = list(train_data.class_indices.keys())
num_classes = len(emotion_labels)

print("✅ Classes:", emotion_labels)


Found 168 images belonging to 7 classes.
Found 168 images belonging to 7 classes.
✅ Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [4]:
from tensorflow.keras.optimizers import Adam

def build_custom_cnn(input_shape=(128, 128, 3), num_classes=7):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ])
    return model

model = build_custom_cnn(input_shape=(128,128,3), num_classes=num_classes)
model.compile(optimizer=Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,289,479 (16.36 MB)

 Trainable params: 4,289,031 (16.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [5]:
EPOCHS = 10

start = time.time()
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    verbose=1
)
print(f"⏱ Training finished in {(time.time()-start)/60:.2f} minutes")


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 109s 10s/step - accuracy: 0.1299 - loss: 4.0471 - val_accuracy: 0.2202 - val_loss: 1.9394
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.1631 - loss: 2.9571 - val_accuracy: 0.1607 - val_loss: 1.9524
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.3713 - loss: 1.8890 - val_accuracy: 0.1429 - val_loss: 2.0581
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.3133 - loss: 1.7957 - val_accuracy: 0.1429 - val_loss: 2.2347
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.3623 - loss: 1.7186 - val_accuracy: 0.1429 - val_loss: 2.4546
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.3738 - loss: 1.7886 - val_accuracy: 0.1429 - val_loss: 2.7325
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.3568 - loss: 1.6496 - val_accuracy: 0.1429 - val_loss: 3.0048
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.2619 - loss: 1.7371 - val_accuracy: 0.1429 - val_los

In [6]:
save_dir = Path("/content/drive/MyDrive/EmotionDetection_Custom")
save_dir.mkdir(parents=True, exist_ok=True)

model_path = save_dir / "CustomCNN_emotion_model.keras"
model.save(model_path)
print(f"💾 Saved model: {model_path}")

metadata = {
    "model_name": "CustomCNN",
    "emotion_labels": emotion_labels,
    "class_indices": train_data.class_indices
}

meta_path = save_dir / "CustomCNN_metadata.pkl"
with open(meta_path, "wb") as f:
    pickle.dump(metadata, f)
print(f"🧾 Saved metadata: {meta_path}")


💾 Saved model: /content/drive/MyDrive/EmotionDetection_Custom/CustomCNN_emotion_model.keras
🧾 Saved metadata: /content/drive/MyDrive/EmotionDetection_Custom/CustomCNN_metadata.pkl


In [11]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

test_img_path = list((val_dir / emotion_labels[5]).glob("*.jpg"))[0]  # any test image
print("Testing on:", test_img_path)

img = load_img(test_img_path, target_size=IMG_SIZE)
arr = img_to_array(img) / 255.0
arr = np.expand_dims(arr, 0)

pred = model.predict(arr)
label = emotion_labels[np.argmax(pred)]
print(f"✅ Prediction: {label}")


Testing on: /content/drive/MyDrive/24_images/test/sad/PrivateTest_1414350.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
✅ Prediction: disgust
